# Team 3: Tim Schurmann, Clarence Robinson, Abby Herrup, & Brett Thompson

# Dependencies

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
from sqlalchemy import create_engine, inspect
import numpy as np
import time
import datetime
from api_keys import api_key
from config import *
#Database connection definitions in config

# Database Connection

In [3]:
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

### View Data in DB

In [4]:
inspector=inspect(engine)
inspector.get_table_names()

['category', 'sources', 'headlines']

In [5]:
for column in inspector.get_columns("sources"):
    print(column)

{'name': 'source_id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('sources_source_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'source_name', 'type': VARCHAR(length=100), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'last_updated', 'type': TIMESTAMP(), 'nullable': False, 'default': 'LOCALTIMESTAMP', 'autoincrement': False, 'comment': None}


# Data Scrape

In [6]:
# URL of page to be scraped
url = 'https://www.allsides.com/unbiased-balanced-news'
response = requests.get(url)
response.status_code

200

In [7]:
html = response.content
soup = bs(html, "lxml")

# Sources 
source = soup.find("div", class_="row-fluid bias-trio-wrapper")
source_name = source.find_all("div", class_="news-source")

# Articles
article = soup.find("div", class_="row-fluid bias-trio-wrapper")
article_name = article.find_all("div", class_="news-title")

# Category
category_name = soup.find_all("div", class_="news-topic")

# Sub Header
article_sub_header = soup.find_all("div", class_="topic-description")

# Source url
article_url = soup.find("div", class_="row-fluid bias-trio-wrapper")
article_urls = article_url.find_all('div', class_="news-title")

In [8]:
print(len(article_urls))
print(len(article_sub_header))
print(len(category_name))
print(len(article_name))
print(len(source_name))

45
45
45
45
45


#### Append Scraped Data to Lists

In [9]:
sources = []
articles = []
categories = []
sub_headers = []
urls = []

for x in source_name:
    source = x.text.strip()
    sources.append(source)
    
for x in article_name:
    article = x.text.strip()
    articles.append(article)
    
for x in category_name:
    category = x.text.strip()
    categories.append(category)

for x in article_sub_header:
    sub_header = x.text.strip()
    sub_headers.append(sub_header)

for x in article_urls:
    urls.append(x.a.get('href'))

In [10]:
print(len(sources))
print(len(articles))
print(len(categories))
print(len(sub_headers))
print(len(urls))

45
45
45
45
45


# API Data Scrape

In [11]:
search=["opinion","technology","politics","economy"]


for news in search:
    api_urls = f"https://content.guardianapis.com/search?q={news}&show-fields=trailText,headline&api-key={api_key}"
    response = requests.get(api_urls)
    data=response.json()
    for x in range (0,10):
        title=data["response"]["results"][x]["fields"]["headline"]
        webUrl=data["response"]["results"][x]["webUrl"]
        subheader=data["response"]["results"][x]["fields"]["trailText"]
        section=data["response"]["results"][x]["sectionName"]
        source_input="The Guardian"
        
        
        articles.append(title)
        sub_headers.append(subheader)
        urls.append(webUrl)
        categories.append(section)
        sources.append(source_input)

In [12]:
print(len(sources))
print(len(articles))
print(len(categories))
print(len(sub_headers))
print(len(urls))

85
85
85
85
85


#### Unique Categories/Sources Data Scrape

In [13]:
categories_unique = []
for x in categories:
    if x not in categories_unique:
        categories_unique.append(x)
print(len(categories_unique))

sources_unique = []
for x in sources:
    if x not in sources_unique:
        sources_unique.append(x)
print(len(sources_unique))

40
37


# DataFrame Creation

### Sources DataFrame

In [14]:
sources_df = pd.DataFrame()
sources_df['source_name'] = sources_unique
sources_df.tail()

,source_name
32,David Harsanyi
33,Fox News (Online News)
34,National Review
35,Bizpac Review
36,Reason


### Category DataFrame

In [15]:
category_df = pd.DataFrame()
category_df['category_name'] = categories_unique
category_df.tail()

,category_name
35,Life and style
36,Environment
37,US news
38,News
39,Money


### API DataFrame

In [16]:
# api_df = pd.DataFrame()
# api_df["article_name"]= article_name
# api_df["article_sub_header"] = article_sub_header
# api_df["url"] = url
# api_df["category"] = category
# api_df["source"] = api_source

# api_df.head()

### Headlines DataFrame

In [17]:
headlines_df = pd.DataFrame()

headlines_df['article_name'] = articles
headlines_df['article_sub_header'] = sub_headers
headlines_df['source_name'] = sources
headlines_df['category_name'] = categories
headlines_df['url'] = urls

headlines_df.tail()

,article_name,article_sub_header,source_name,category_name,url
80,Why low inflation is a worrying sign for the U...,<strong>Analysis</strong>: lower prices may ha...,The Guardian,Business,https://www.theguardian.com/business/2020/dec/...
81,Secondhand no longer second-best for UK's 'cir...,"During lockdown, the thoughts of many have tur...",The Guardian,Money,https://www.theguardian.com/money/2020/nov/22/...
82,Covid-hit Merseyside economy 'may collapse wit...,Local leaders say financial support needed whe...,The Guardian,World news,https://www.theguardian.com/world/2020/sep/30/...
83,How the Great Frost of 1709 left England’s eco...,Three hundred years ago it was a three-month c...,The Guardian,News,https://www.theguardian.com/news/2020/dec/17/h...
84,Families ‘facing hardest period in five decade...,Household income will rise by just £220 in fiv...,The Guardian,Business,https://www.theguardian.com/business/2020/nov/...


# Uploading to SQL DB

### Sources Table

In [18]:
conn = engine.connect()
sources_df.to_sql("sources", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

#### Save to df

In [19]:
query = """
        SELECT
            *
        FROM
            sources
        """

conn = engine.connect()
sql_sources_df = pd.read_sql(query, con=conn)
conn.close()

sql_sources_df.tail()

,source_id,source_name,last_updated
32,33,David Harsanyi,2020-12-22 01:33:45.433547
33,34,Fox News (Online News),2020-12-22 01:33:45.433547
34,35,National Review,2020-12-22 01:33:45.433547
35,36,Bizpac Review,2020-12-22 01:33:45.433547
36,37,Reason,2020-12-22 01:33:45.433547


### Category Table

In [20]:
conn = engine.connect()
category_df.to_sql("category", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

#### Save to df

In [21]:
query = """
        SELECT
            *
        FROM
            category
        """
#Save to Df
conn = engine.connect()
sql_cat_df = pd.read_sql(query, con=conn)
conn.close()

sql_cat_df.tail()

,category_id,category_name,last_updated
35,36,Life and style,2020-12-22 01:33:45.701341
36,37,Environment,2020-12-22 01:33:45.701341
37,38,US news,2020-12-22 01:33:45.701341
38,39,News,2020-12-22 01:33:45.701341
39,40,Money,2020-12-22 01:33:45.701341


### Headlines Table

In [22]:
source_ids = []
category_ids = []

for x in headlines_df.source_name:
    source_ids.append(sql_sources_df.source_id.loc[sql_sources_df['source_name'] == x].values[0])
    
for x in headlines_df.category_name:
    category_ids.append(sql_cat_df.category_id.loc[sql_cat_df['category_name'] == x].values[0])

#### Save to df

In [23]:
sql_headlines_df = pd.DataFrame()

sql_headlines_df['article_name'] = articles
sql_headlines_df['article_sub_header'] = sub_headers
sql_headlines_df['source_id'] = source_ids
sql_headlines_df['category_id'] = category_ids
sql_headlines_df['url'] = urls

sql_headlines_df.tail()

,article_name,article_sub_header,source_id,category_id,url
80,Why low inflation is a worrying sign for the U...,<strong>Analysis</strong>: lower prices may ha...,12,7,https://www.theguardian.com/business/2020/dec/...
81,Secondhand no longer second-best for UK's 'cir...,"During lockdown, the thoughts of many have tur...",12,40,https://www.theguardian.com/money/2020/nov/22/...
82,Covid-hit Merseyside economy 'may collapse wit...,Local leaders say financial support needed whe...,12,28,https://www.theguardian.com/world/2020/sep/30/...
83,How the Great Frost of 1709 left England’s eco...,Three hundred years ago it was a three-month c...,12,39,https://www.theguardian.com/news/2020/dec/17/h...
84,Families ‘facing hardest period in five decade...,Household income will rise by just £220 in fiv...,12,7,https://www.theguardian.com/business/2020/nov/...


In [24]:
conn = engine.connect()
sql_headlines_df.to_sql("headlines", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

In [25]:
query = """
        SELECT
            *
        FROM
            headlines
        """
#Save to Df
conn = engine.connect()
sql_headlines_df = pd.read_sql(query, con=conn)
conn.close()

sql_headlines_df.tail()

,article_id,article_name,article_sub_header,source_id,category_id,url,last_updated
80,81,Why low inflation is a worrying sign for the U...,<strong>Analysis</strong>: lower prices may ha...,12,7,https://www.theguardian.com/business/2020/dec/...,2020-12-22 01:33:46.139543
81,82,Secondhand no longer second-best for UK's 'cir...,"During lockdown, the thoughts of many have tur...",12,40,https://www.theguardian.com/money/2020/nov/22/...,2020-12-22 01:33:46.139543
82,83,Covid-hit Merseyside economy 'may collapse wit...,Local leaders say financial support needed whe...,12,28,https://www.theguardian.com/world/2020/sep/30/...,2020-12-22 01:33:46.139543
83,84,How the Great Frost of 1709 left England’s eco...,Three hundred years ago it was a three-month c...,12,39,https://www.theguardian.com/news/2020/dec/17/h...,2020-12-22 01:33:46.139543
84,85,Families ‘facing hardest period in five decade...,Household income will rise by just £220 in fiv...,12,7,https://www.theguardian.com/business/2020/nov/...,2020-12-22 01:33:46.139543


# Updating SQL DB

## Sources Table SQL

In [26]:
# Pull sources Table from SQL db
query = """
        SELECT
            *
        FROM
            sources
        """
conn = engine.connect()
sql_sources_df = pd.read_sql(query, con=conn)
conn.close()

sql_sources_df.tail()

,source_id,source_name,last_updated
32,33,David Harsanyi,2020-12-22 01:33:45.433547
33,34,Fox News (Online News),2020-12-22 01:33:45.433547
34,35,National Review,2020-12-22 01:33:45.433547
35,36,Bizpac Review,2020-12-22 01:33:45.433547
36,37,Reason,2020-12-22 01:33:45.433547


In [27]:
# Pull current list of Source (we are avoiding duplicating the list here)
sql_sources_list = sql_sources_df.source_name.to_list()

# Create empty list for updates
sources_update = []

# Loop through current list of Sources
for source in sources_df.source_name:
    # If source is NOT IN current list... Append to our update list
    if source not in sql_sources_list:
        sources_update.append(source)

source_adds = len(sources_update)
print(f"The number of sources to be added to SQL is: {source_adds}")

# Save updates in a new DataFrame
sources_update_df = pd.DataFrame()
sources_update_df['source_name'] = sources_update
sources_update_df.head()

The number of sources to be added to SQL is: 0


,source_name


#### Upload new Sources to Sources Table in SQL

In [28]:
# Send UPDATES DataFrame to SQL
# Any NEW sources will be added to the end of the table
conn = engine.connect()
sources_update_df.to_sql("sources", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

## Category Table SQL

In [29]:
query = """
        SELECT
            *
        FROM
            category
        """
conn = engine.connect()
sql_cat_df = pd.read_sql(query, con=conn)
conn.close()

sql_cat_df.tail()

,category_id,category_name,last_updated
35,36,Life and style,2020-12-22 01:33:45.701341
36,37,Environment,2020-12-22 01:33:45.701341
37,38,US news,2020-12-22 01:33:45.701341
38,39,News,2020-12-22 01:33:45.701341
39,40,Money,2020-12-22 01:33:45.701341


In [30]:
# Pull current list of Categories (we are avoiding duplicating the list here)
sql_cat_list = sql_cat_df.category_name.to_list()

# Create empty list for updates
cats_update = []

# Loop through current list of Categories
for cat in category_df.category_name:
    # If category is NOT IN current list... Append to our update list
    if cat not in sql_cat_list:
        cats_update.append(cat)

cat_adds = len(cats_update)
print(f"The number of sources to be added to SQL is: {cat_adds}")

# Save updates in a new DataFrame
cat_update_df = pd.DataFrame()
cat_update_df['category_name'] = cats_update
cat_update_df.head()

The number of sources to be added to SQL is: 0


,category_name


#### Upload new categories to Category Table in SQL

In [31]:
conn = engine.connect()
cat_update_df.to_sql("category", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

## Headlines Table SQL

In [32]:
query = """
        SELECT
            *
        FROM
            headlines
        """
conn = engine.connect()
sql_headlines_df = pd.read_sql(query, con=conn)
conn.close()

sql_headlines_df.head()

,article_id,article_name,article_sub_header,source_id,category_id,url,last_updated
0,1,The US is on the verge of the biggest anti-mon...,ANALYSIS If you’re a corrupt foreign official ...,1,1,https://www.allsides.com/news/2020-12-21-1231/...,2020-12-22 01:33:46.139543
1,2,Armed Anti-Lockdown Protesters Just Tried to S...,"Right-wing protesters, including many armed wi...",2,2,https://www.allsides.com/news/2020-12-21-1209/...,2020-12-22 01:33:46.139543
2,3,"As census deadline looms, experts warn rushing...",The fate of this year's census remains uncerta...,3,3,https://www.allsides.com/news/2020-12-21-1144/...,2020-12-22 01:33:46.139543
3,4,Why America can’t rely solely on individuals t...,ANALYSIS The countries that defeated the coron...,1,4,https://www.allsides.com/news/2020-12-21-0850/...,2020-12-22 01:33:46.139543
4,5,Trump Is Losing His Mind,OPINION The president is discussing martial la...,4,5,https://www.allsides.com/news/2020-12-21-0725/...,2020-12-22 01:33:46.139543


In [33]:
sql_headlines_list = sql_headlines_df.article_name.to_list()

headlines_update = []
count = 0

# Loop through current list of Headlines
for article in headlines_df.article_name:
    
    #If there is a new headline that does not match our SQL list then....
    if article not in sql_headlines_list:
        
        #Save that headline to an empty update list
        headlines_update.append(article)
        
        #If there IS something in the update list then....
        if len(headlines_update) > 0:
            
            #Find the article
            update_df = headlines_df.loc[headlines_df.article_name == article]
            headline_updater = update_df.article_name.values[0]
            sub_header_update = update_df.article_sub_header.values[0]
            source_id_update = sql_sources_df.source_id.loc[sql_sources_df['source_name'] == update_df.source_name.values[0]]
            category_id_update = sql_cat_df.category_id.loc[sql_cat_df['category_name'] == update_df.category_name.values[0]]
            url_updater = update_df.url.values[0]
            
            #Save variables off to DataFrame
            updater_df = pd.DataFrame()

            updater_df['article_name'] = headline_updater
            updater_df['article_sub_header'] = sub_header_update
            updater_df['source_id'] = source_id_update
            updater_df['category_id'] = category_id_update
            updater_df['url'] = url_updater
            
            #Load to SQL
            conn = engine.connect()
            updater_df.to_sql("headlines", schema="public", con=conn, 
                                          if_exists="append", index=False, method="multi")
            conn.close()

            print(f"Added {article} to SQL headlines table")
            headlines_update = []
    else: 
          count = count + 1
          print(f"Processing headline: {count} of {len(headlines_df.article_name)}")

Processing headline: 1 of 85
Processing headline: 2 of 85
Processing headline: 3 of 85
Processing headline: 4 of 85
Processing headline: 5 of 85
Processing headline: 6 of 85
Processing headline: 7 of 85
Processing headline: 8 of 85
Processing headline: 9 of 85
Processing headline: 10 of 85
Processing headline: 11 of 85
Processing headline: 12 of 85
Processing headline: 13 of 85
Processing headline: 14 of 85
Processing headline: 15 of 85
Processing headline: 16 of 85
Processing headline: 17 of 85
Processing headline: 18 of 85
Processing headline: 19 of 85
Processing headline: 20 of 85
Processing headline: 21 of 85
Processing headline: 22 of 85
Processing headline: 23 of 85
Processing headline: 24 of 85
Processing headline: 25 of 85
Processing headline: 26 of 85
Processing headline: 27 of 85
Processing headline: 28 of 85
Processing headline: 29 of 85
Processing headline: 30 of 85
Processing headline: 31 of 85
Processing headline: 32 of 85
Processing headline: 33 of 85
Processing headline